# Network analysis of covert social network for the Madoff Fraud

Import the necessary modules

In [1]:
import pandas as pd
import numpy as np
import urllib.request as urllib
from io import BytesIO
from zipfile import ZipFile
import networkx as nx
import matplotlib.pyplot as plt

Provide the link where the file is available

In [2]:
url = 'http://www.casos.cs.cmu.edu/tools/datasets/external/madoff/madoff.zip'

Open the Zip file, which contains a csv file

In [3]:
with urllib.urlopen(url) as stream:
    with ZipFile(BytesIO(stream.read())) as archive:
        archive.printdir()
        txt = archive.read('MADOFF.csv').decode()

File Name                                             Modified             Size
MADOFF.csv                                     2017-01-08 00:46:22        13372


convert the string into comma separated values

In [4]:
comma_sep = txt.split(',')

examine the structure of the data

In [5]:
print(comma_sep[:123])

['\ufeff""', 'HSBC_Holdings', 'Genevalor_Benbassat', 'Phoenix_Holdings', 'Thema_Fund', 'Herald_Lux_Fund', 'Capital_Bank_Austria', 'Cohmad_securities', 'Bank_Medici', 'UniCredit_SpA', 'Pioneer_Alt_Investments', 'Rothschild_Cie', 'Access_Int_Advisors', 'BNP_Paribas', 'EIM_Group', 'Nipponkoa_Insurance', 'Mirabaud_Cie', 'Zeus_Partners_Ltd', 'Banco_Safra', 'CMG', 'Brighton_Co', 'Avellino_Bienes', 'Neu_Privat_Bank', 'Banco_Santander', 'Optimal_Strategic_US', 'Sumitomo_Life_Ins', 'Fukoku_Mutual_Life', 'Fix_Asset_Mgt', 'Fairfield_Greenwich', 'Nordea_Bank_AB', 'Banque_Benedict_HentchCie', 'Genium_Advisors', 'Great_Eastern_Holdings', 'Prospect_Capital', 'Union_Bancaire_Privee', 'Sterling_Equities', 'Nomura_Holdings', 'Stanford_Capital_Mgt', 'FIM_Advisers', 'M&B_Capital_Advisors', 'Man_Group_PLC', 'Notz_Stucki_Cie', 'Credicorp_Ltd', 'S&P_Investment', 'EFG_International_AG', 'Maxam_Absolute_Return_Fund', 'Tremont_Group_Holdings', 'Opperheimer_Funds', 'Bradean_Alternatives_Lts', 'Rye_Investment_Fun

confirm the length of each row if the values were placed in a table

In [6]:
print(comma_sep[-62:])

['""\r\nBernard_Madoff_Investment', '""', '""', '""', '""', '""', '""', '""', '""', '""', '""', '""', '""', '""', '""', '""', '""', '""', '""', '""', '""', '""', '""', '""', '""', '""', '""', '""', '""', '""', '""', '""', '""', '""', '""', '""', '""', '""', '""', '""', '""', '""', '""', '""', '""', '""', '""', '""', '""', '""', '""', '""', '""', '""', '""', '""', '""', '""', '""', '""', '""', '""\r\n']


In [7]:
(len(comma_sep)-1)/61

62.0

convert the end of row marker into a comma

In [8]:
fixed = txt.replace("\r\n", ",")

inspect the data in its new format

In [9]:
csv_file = fixed.split(',')

In [10]:
print(csv_file[:123])

['\ufeff""', 'HSBC_Holdings', 'Genevalor_Benbassat', 'Phoenix_Holdings', 'Thema_Fund', 'Herald_Lux_Fund', 'Capital_Bank_Austria', 'Cohmad_securities', 'Bank_Medici', 'UniCredit_SpA', 'Pioneer_Alt_Investments', 'Rothschild_Cie', 'Access_Int_Advisors', 'BNP_Paribas', 'EIM_Group', 'Nipponkoa_Insurance', 'Mirabaud_Cie', 'Zeus_Partners_Ltd', 'Banco_Safra', 'CMG', 'Brighton_Co', 'Avellino_Bienes', 'Neu_Privat_Bank', 'Banco_Santander', 'Optimal_Strategic_US', 'Sumitomo_Life_Ins', 'Fukoku_Mutual_Life', 'Fix_Asset_Mgt', 'Fairfield_Greenwich', 'Nordea_Bank_AB', 'Banque_Benedict_HentchCie', 'Genium_Advisors', 'Great_Eastern_Holdings', 'Prospect_Capital', 'Union_Bancaire_Privee', 'Sterling_Equities', 'Nomura_Holdings', 'Stanford_Capital_Mgt', 'FIM_Advisers', 'M&B_Capital_Advisors', 'Man_Group_PLC', 'Notz_Stucki_Cie', 'Credicorp_Ltd', 'S&P_Investment', 'EFG_International_AG', 'Maxam_Absolute_Return_Fund', 'Tremont_Group_Holdings', 'Opperheimer_Funds', 'Bradean_Alternatives_Lts', 'Rye_Investment_Fun

confirm the number of the rows

In [11]:
rows = (len(csv_file)-1)/62
print(rows)

62.0


inspect the values that will make up the header row

In [12]:
csv_file[0:62]

['\ufeff""',
 'HSBC_Holdings',
 'Genevalor_Benbassat',
 'Phoenix_Holdings',
 'Thema_Fund',
 'Herald_Lux_Fund',
 'Capital_Bank_Austria',
 'Cohmad_securities',
 'Bank_Medici',
 'UniCredit_SpA',
 'Pioneer_Alt_Investments',
 'Rothschild_Cie',
 'Access_Int_Advisors',
 'BNP_Paribas',
 'EIM_Group',
 'Nipponkoa_Insurance',
 'Mirabaud_Cie',
 'Zeus_Partners_Ltd',
 'Banco_Safra',
 'CMG',
 'Brighton_Co',
 'Avellino_Bienes',
 'Neu_Privat_Bank',
 'Banco_Santander',
 'Optimal_Strategic_US',
 'Sumitomo_Life_Ins',
 'Fukoku_Mutual_Life',
 'Fix_Asset_Mgt',
 'Fairfield_Greenwich',
 'Nordea_Bank_AB',
 'Banque_Benedict_HentchCie',
 'Genium_Advisors',
 'Great_Eastern_Holdings',
 'Prospect_Capital',
 'Union_Bancaire_Privee',
 'Sterling_Equities',
 'Nomura_Holdings',
 'Stanford_Capital_Mgt',
 'FIM_Advisers',
 'M&B_Capital_Advisors',
 'Man_Group_PLC',
 'Notz_Stucki_Cie',
 'Credicorp_Ltd',
 'S&P_Investment',
 'EFG_International_AG',
 'Maxam_Absolute_Return_Fund',
 'Tremont_Group_Holdings',
 'Opperheimer_Funds',


create an empty dataframe to hold the rows

In [13]:
df = pd.DataFrame(columns=csv_file[0:62])

In [14]:
df.shape

(0, 62)

load the dataframe with the rows from the data

In [15]:
start = 62
end = 124
for num in range(int(rows)-1):
    row = csv_file[start:end]
    df.loc[len(df)] = row
    start += 62
    end += 62

rename the first column

In [16]:
names = df.columns.tolist()
names[0] = 'firm1'
df.columns = names

inspect the new dataframe

In [17]:
df.head()

,firm1,HSBC_Holdings,Genevalor_Benbassat,Phoenix_Holdings,Thema_Fund,Herald_Lux_Fund,Capital_Bank_Austria,Cohmad_securities,Bank_Medici,UniCredit_SpA,...,Banco_Bilbao_Vizcaya_Argentaria,Kingate_Global_Fund,Kingate_Euro_Fund,Ascot_Partners,Gabriel_Capital,Sterling_Stamos_Capital_Mgt,Spring_Mountain_Capital,Ariel_Capital,Fortis_Bank_Nederland,Bernard_Madoff_Investment
0,HSBC_Holdings,"""""","""""","""""","""""",1,"""""","""""","""""","""""",...,"""""","""""","""""","""""","""""","""""","""""","""""","""""",""""""
1,Genevalor_Benbassat,"""""","""""","""""",1,"""""","""""","""""","""""","""""",...,"""""","""""","""""","""""","""""","""""","""""","""""","""""",""""""
2,Phoenix_Holdings,"""""","""""","""""",1,"""""","""""","""""","""""","""""",...,"""""","""""","""""","""""","""""","""""","""""","""""","""""",""""""
3,Thema_Fund,"""""","""""","""""","""""","""""","""""","""""",1,"""""",...,"""""","""""","""""","""""","""""","""""","""""","""""","""""",""""""
4,Herald_Lux_Fund,"""""","""""","""""","""""","""""","""""","""""",1,"""""",...,"""""","""""","""""","""""","""""","""""","""""","""""","""""",""""""


In [18]:
df.tail()

,firm1,HSBC_Holdings,Genevalor_Benbassat,Phoenix_Holdings,Thema_Fund,Herald_Lux_Fund,Capital_Bank_Austria,Cohmad_securities,Bank_Medici,UniCredit_SpA,...,Banco_Bilbao_Vizcaya_Argentaria,Kingate_Global_Fund,Kingate_Euro_Fund,Ascot_Partners,Gabriel_Capital,Sterling_Stamos_Capital_Mgt,Spring_Mountain_Capital,Ariel_Capital,Fortis_Bank_Nederland,Bernard_Madoff_Investment
56,Sterling_Stamos_Capital_Mgt,"""""","""""","""""","""""","""""","""""","""""","""""","""""",...,"""""","""""","""""","""""",1,"""""","""""","""""","""""",""""""
57,Spring_Mountain_Capital,"""""","""""","""""","""""","""""","""""","""""","""""","""""",...,"""""","""""","""""","""""",1,"""""","""""","""""","""""",""""""
58,Ariel_Capital,"""""","""""","""""","""""","""""","""""","""""","""""","""""",...,"""""","""""","""""","""""",1,"""""","""""","""""","""""",""""""
59,Fortis_Bank_Nederland,"""""","""""","""""","""""","""""","""""","""""","""""","""""",...,"""""","""""","""""","""""","""""","""""","""""",1,"""""",""""""
60,Bernard_Madoff_Investment,"""""","""""","""""","""""","""""","""""","""""","""""","""""",...,"""""","""""","""""","""""","""""","""""","""""","""""","""""",""""""


transform the dataframe into three columns, to show the relationships between firms

In [19]:
melted_df = df.melt('firm1', var_name='firm2', value_name='relationship')

convert the relationship into a binary flag

In [20]:
melted_df['relationship'] = pd.to_numeric(melted_df['relationship'], errors='coerce')

In [21]:
melted_df = melted_df.replace(np.nan,0)

In [22]:
melted_df.head()

,firm1,firm2,relationship
0,HSBC_Holdings,HSBC_Holdings,0.0
1,Genevalor_Benbassat,HSBC_Holdings,0.0
2,Phoenix_Holdings,HSBC_Holdings,0.0
3,Thema_Fund,HSBC_Holdings,0.0
4,Herald_Lux_Fund,HSBC_Holdings,0.0


In [23]:
melted_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3721 entries, 0 to 3720
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   firm1         3721 non-null   object 
 1   firm2         3721 non-null   object 
 2   relationship  3721 non-null   float64
dtypes: float64(1), object(2)
memory usage: 87.3+ KB


In [24]:
melted_df.describe()

,relationship
count,3721.000000
mean,0.016393
std,0.127000
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


look for patterns in the sending firm

In [78]:
senders = melted_df.groupby('firm1').sum().reset_index()
senders.columns = ['firm1','relationships']
senders.sort_values('relationships', ascending=False)

C:\Users\Sterling\AppData\Local\Temp\ipykernel_16856\2082119920.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  senders = melted_copy.groupby('firm1').sum().reset_index()


,firm1,relationships
16,Cohmad_securities,2.0
0,Access_Int_Advisors,1.0
33,M&B_Capital_Advisors,1.0
34,Man_Group_PLC,1.0
35,Maxam_Absolute_Return_Fund,1.0
...,...,...
26,Genevalor_Benbassat,1.0
27,Genium_Advisors,1.0
28,Great_Eastern_Holdings,1.0
60,Zeus_Partners_Ltd,1.0


preview any patterns in the receiving firm

In [26]:
receivers = melted_df.groupby('firm2').sum().reset_index()
receivers.columns = ['firm2','relationships']
receivers.sort_values('relationships', ascending=False)

C:\Users\Sterling\AppData\Local\Temp\ipykernel_16856\2716391413.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  receivers = melted_df.groupby('firm2').sum().reset_index()


,firm2,relationships
11,Bernard_Madoff_Investment,27.0
9,Bank_Medici,6.0
21,Fairfield_Greenwich,6.0
25,Gabriel_Capital,4.0
57,Tremont_Group_Holdings,3.0
...,...,...
6,Banco_Bilbao_Vizcaya_Argentaria,0.0
33,M&B_Capital_Advisors,0.0
34,Man_Group_PLC,0.0
35,Maxam_Absolute_Return_Fund,0.0


In [27]:
melted_df['relationship'].values.sum()

61.0

In [28]:
melted_sorted = melted_df.sort_values('relationship', ascending=False)

In [29]:
melted_sorted.head(62)

,firm1,firm2,relationship
3684,Sumitomo_Life_Ins,Bernard_Madoff_Investment,1.0
3676,Zeus_Partners_Ltd,Bernard_Madoff_Investment,1.0
1680,Union_Bancaire_Privee,Fairfield_Greenwich,1.0
3671,Access_Int_Advisors,Bernard_Madoff_Investment,1.0
3672,BNP_Paribas,Bernard_Madoff_Investment,1.0
...,...,...,...
2791,Opperheimer_Funds,Tremont_Group_Holdings,1.0
3695,Nomura_Holdings,Bernard_Madoff_Investment,1.0
2789,Maxam_Absolute_Return_Fund,Tremont_Group_Holdings,1.0
3699,Man_Group_PLC,Bernard_Madoff_Investment,1.0


take the rows with positive values in the relationship column and put them in a separate dataframe

In [30]:
graph_df = melted_sorted.iloc[:61]

In [31]:
graph_df.head()

,firm1,firm2,relationship
3684,Sumitomo_Life_Ins,Bernard_Madoff_Investment,1.0
3676,Zeus_Partners_Ltd,Bernard_Madoff_Investment,1.0
1680,Union_Bancaire_Privee,Fairfield_Greenwich,1.0
3671,Access_Int_Advisors,Bernard_Madoff_Investment,1.0
3672,BNP_Paribas,Bernard_Madoff_Investment,1.0


In [32]:
graph_df.tail()

,firm1,firm2,relationship
2310,Kingate_Euro_Fund,FIM_Advisers,1.0
2791,Opperheimer_Funds,Tremont_Group_Holdings,1.0
3695,Nomura_Holdings,Bernard_Madoff_Investment,1.0
2789,Maxam_Absolute_Return_Fund,Tremont_Group_Holdings,1.0
3699,Man_Group_PLC,Bernard_Madoff_Investment,1.0


create the network graph

In [69]:
G = nx.from_pandas_edgelist(graph_df, 'firm1', 'firm2', edge_attr = True)

get summary statistics about the graph

In [70]:
nx.density(G)

0.03333333333333333

In [71]:
nx.number_of_edges(G)

61

"figure out the most effective node to remove," per the assignment instructions

In [72]:
btwn_cent = sorted(nx.betweenness_centrality(G).items(), key=lambda x:x[1], reverse = True)
print(btwn_cent)

[('Bernard_Madoff_Investment', 0.936723163841808), ('Cohmad_securities', 0.2593220338983051), ('Bank_Medici', 0.2480225988700565), ('Fairfield_Greenwich', 0.19152542372881357), ('Tremont_Group_Holdings', 0.188135593220339), ('Gabriel_Capital', 0.16045197740112996), ('Rye_Investment_Funds', 0.09830508474576272), ('FIM_Advisers', 0.09774011299435029), ('Thema_Fund', 0.06610169491525424), ('Zeus_Partners_Ltd', 0.03333333333333333), ('Access_Int_Advisors', 0.03333333333333333), ('Brighton_Co', 0.03333333333333333), ('Banco_Santander', 0.03333333333333333), ('Kingate_Global_Fund', 0.03333333333333333), ('Ariel_Capital', 0.03333333333333333), ('Herald_Lux_Fund', 0.03333333333333333), ('Pioneer_Alt_Investments', 0.03333333333333333), ('Sumitomo_Life_Ins', 0.0), ('Union_Bancaire_Privee', 0.0), ('BNP_Paribas', 0.0), ('EIM_Group', 0.0), ('Nipponkoa_Insurance', 0.0), ('Mirabaud_Cie', 0.0), ('Rothschild_Cie', 0.0), ('Great_Eastern_Holdings', 0.0), ('Avellino_Bienes', 0.0), ('Neu_Privat_Bank', 0.0)

remove the most effective node

In [73]:
G.remove_node(btwn_cent[0][0])

note the change in the summary statistics, which show a lower density, reduced number of edges, and lower betweenness centrality

In [74]:
nx.density(G)

0.0192090395480226

In [75]:
nx.number_of_edges(G)

34

In [76]:
btwn_cent = sorted(nx.betweenness_centrality(G).items(), key=lambda x:x[1], reverse = True)
print(btwn_cent)

[('Bank_Medici', 0.018118059614260665), ('Fairfield_Greenwich', 0.008766803039158387), ('Thema_Fund', 0.008766803039158387), ('Rye_Investment_Funds', 0.007013442431326709), ('Gabriel_Capital', 0.0052600818234950315), ('Tremont_Group_Holdings', 0.0052600818234950315), ('Herald_Lux_Fund', 0.004675628287551139), ('Pioneer_Alt_Investments', 0.004675628287551139), ('Ariel_Capital', 0.0023378141437755697), ('Kingate_Global_Fund', 0.0011689070718877848), ('FIM_Advisers', 0.0011689070718877848), ('Sumitomo_Life_Ins', 0.0), ('Zeus_Partners_Ltd', 0.0), ('Union_Bancaire_Privee', 0.0), ('Access_Int_Advisors', 0.0), ('BNP_Paribas', 0.0), ('EIM_Group', 0.0), ('Nipponkoa_Insurance', 0.0), ('Mirabaud_Cie', 0.0), ('Rothschild_Cie', 0.0), ('Great_Eastern_Holdings', 0.0), ('Brighton_Co', 0.0), ('Avellino_Bienes', 0.0), ('Neu_Privat_Bank', 0.0), ('Banco_Santander', 0.0), ('Optimal_Strategic_US', 0.0), ('Prospect_Capital', 0.0), ('Genium_Advisors', 0.0), ('Fix_Asset_Mgt', 0.0), ('Meridian_Capital_Partners'